## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-22-20-44-59 +0000,startribune,Prep Bowl: Live updates from the football stat...,https://www.startribune.com/live-scores-and-ch...
1,2025-11-22-20-43-10 +0000,nypost,News of the World: What you missed this week i...,https://nypost.com/2025/11/22/world-news/news-...
2,2025-11-22-20-39-00 +0000,wsj,Boston Fed President says she sees no strong n...,https://www.wsj.com/economy/central-banking/fe...
3,2025-11-22-20-36-42 +0000,nyt,"As Democrats Look to a New Generation, John La...",https://www.nytimes.com/2025/11/22/us/politics...
4,2025-11-22-20-34-00 +0000,wsj,President Trump said he could be open to chang...,https://www.wsj.com/politics/policy/trump-says...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,49
48,ukraine,16
40,he,15
426,mamdani,14
45,plan,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
183,2025-11-22-01-00-00 +0000,wsj,President Trump said this week he expects lowe...,https://www.wsj.com/economy/central-banking/fe...,143
4,2025-11-22-20-34-00 +0000,wsj,President Trump said he could be open to chang...,https://www.wsj.com/politics/policy/trump-says...,135
14,2025-11-22-19-44-00 +0000,wsj,President Trump said Friday he was immediately...,https://www.wsj.com/politics/policy/trump-says...,120
105,2025-11-22-12-15-08 +0000,nypost,Trump slams ‘traitor’ Rep. Marjorie Taylor Gre...,https://nypost.com/2025/11/22/us-news/trump-ce...,110
235,2025-11-21-21-34-00 +0000,wsj,Trump Says He Wants Ukraine’s Answer on Peace ...,https://www.wsj.com/world/zelensky-russia-ukra...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
183,143,2025-11-22-01-00-00 +0000,wsj,President Trump said this week he expects lowe...,https://www.wsj.com/economy/central-banking/fe...
178,70,2025-11-22-01-18-00 +0000,wsj,Texas asked the Supreme Court to let it use a ...,https://www.wsj.com/politics/elections/texas-a...
4,64,2025-11-22-20-34-00 +0000,wsj,President Trump said he could be open to chang...,https://www.wsj.com/politics/policy/trump-says...
63,46,2025-11-22-15-51-00 +0000,wsj,"Republican Marjorie Taylor Greene, who fought ...",https://www.wsj.com/politics/policy/marjorie-t...
237,45,2025-11-21-21-33-52 +0000,nypost,Trump says Zohran Mamdani is not a ‘jihadist’ ...,https://nypost.com/2025/11/21/us-news/trump-br...
14,40,2025-11-22-19-44-00 +0000,wsj,President Trump said Friday he was immediately...,https://www.wsj.com/politics/policy/trump-says...
72,37,2025-11-22-15-02-26 +0000,nypost,Ryan Lizza reveals raunchy ‘poem’ RFK Jr. alle...,https://nypost.com/2025/11/22/us-news/ryan-liz...
33,35,2025-11-22-18-21-26 +0000,nypost,Anna Kepner’s mom found out about daughter’s c...,https://nypost.com/2025/11/22/us-news/anna-kep...
49,32,2025-11-22-16-33-16 +0000,bbc,Canada Post and union reach deal in principle ...,https://www.bbc.com/news/articles/cvgkeyl78rwo...
110,28,2025-11-22-11-32-39 +0000,bbc,Nigeria sees one of worst mass abductions as 3...,https://www.bbc.com/news/articles/c3w7621xypyo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
